# Convolutional Neural Networks

This notebook introduces convolutional neural networks (CNNs), a more powerful classification model similar to the Neural Bag-of-Words (BOW) model you explored earlier.

## Outline

- **Part (a):** Model Architecture
- **Part (b):** Implementing the CNN Model
- **Part (c):** Tuning

In [1]:
from __future__ import division
import os, sys, re, json, time, datetime, shutil
import itertools, collections
from importlib import reload
from IPython.display import display, HTML

# NLTK for NLP utils and corpora
import nltk

# NumPy and TensorFlow
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

# Helper libraries
from w266_common import utils, vocabulary, tf_embed_viz, treeviz
from w266_common import patched_numpy_io
# Code for this assignment
import sst

# Monkey-patch NLTK with better Tree display that works on Cloud or other display-less server.
print("Overriding nltk.tree.Tree pretty-printing to use custom GraphViz.")
treeviz.monkey_patch(nltk.tree.Tree, node_style_fn=sst.sst_node_style, format='svg')

Overriding nltk.tree.Tree pretty-printing to use custom GraphViz.


## (a) Model Architecture

CNNs are a more sophisticated neural model for sentence classification than the Neural BOW model we saw in the last section. CNNs operate by sweeping a collection of filters over a text. Each filter produces a sequence of feature values known as a _feature map_. In one of the most basic formulations introduced by [Kim (2014)](https://www.aclweb.org/anthology/D14-1181), a single layer of _pooling_ is used to summarize the _feature maps_ as a fixed length vector. The fixed length vector is then feed to the output layer in order to produce classification labels. A popular choice for the pooling operation is to take the maximum feature value from by each _feature map_.

![Convolutional Neural Network from Kim 2014](kim_2014_figure_1_cnn.png)
*CNN model architure, Figure 1 from Kim (2014)*

We'll use the following notation:
- $h$: filter/kernel length (in words)
- $w^{(i)} \in \mathbb{Z}$, the word id for the $i^{th}$ word of the sequence (as an integer index)
- $x^{(i)} \in \mathbb{R}^d$ for the vector representation (embedding) of $w^{(i)}$
- $x^{i:i+j}$ is the concatenation of $x^{(i)}, x^{(i+1)} ... x^{(i+j)}$ 
- $c^{(i)}_{k}$ is the value of the $k^{th}$ feature map at the $i^{th}$ position along the word sequence, each filter applies over a window of $h$ words and uses non-linearity $f$.
- $c_k$ is one feature map (the $k_{th}$).  Its values are $c^{(0)}_{k}$, $c^{(1)}_{k}$, $c^{(2)}_{k}$, ...
- $\hat{c}_{k}$ is the value of the $k^{th}$ feature after pooling the feature map over the whole sequence.
- $\hat{C}$ is the concatenation of pooled feature maps. 
- $y$ for the target label ($\in 1,\ldots,\mathtt{num\_classes}$)

Our model is defined as:
- **Embedding layer:** $x^{(i)} = W_{embed}[w^{(i)}]$
- **Convolutional layer:** $c^{(i)}_{k} = f(x^{i:i+h-1} W_k + b)$
- **Pooling layer:**  $\hat{c}_{k}$ = $max(c_k)$ = $max(c^{(0)}_{k}, c^{(1)}_{k}...)$ 
- **Output layer:** $\hat{y} = \hat{P}(y) = \mathrm{softmax}(\hat{C} W_{out} + b_{out})$


We'll refer to the first part of this model (**Embedding layer**, **Convolutional layer**, and **Pooling layer**) as the **Encoder**: it has the role of encoding the input sequence into a fixed-length vector representation that we pass to the output layer.

We'll also use these as shorthand for important dimensions:
- `V`: the vocabulary size (equal to `ds.vocab.size`)
- `N`: the maximum number of tokens in the input text
- `embed_dim`: the embedding dimension $d$
- `filters`: number filters per filter length
- `num_classes`: the number of target classes (2 for the binary task)

## (a) Short Answer Questions

When answering these questions in the answers file,
`embed_dim = 10`, `kernel_size = 3`, `filters=128`, `N=10` and `num_classes = 7`.  Assume a single example (no batching).

1. In terms of these values, the vocabulary size `V` and the maximum sequence length `N`, what are the
   shapes of the following variables: 
   $c^{(i)}_{k}$, $c_{k}$, $\hat{c}_{k}$, and $\hat{C}$. Assume a stride size of 1. Assume padding is not used (e.g., for tf.nn.max_pool and tf.nn.conv1d, setting padding='VALID'), provide the shapes listed above.
<p>
2. What are the shapes of $c_{k}$ and $\hat{c}_{k}$ when padding is used.
      (e.g., for tf.nn.max_pool and tf.nn.conv1d, setting padding='same').
<p>
3. How many parameters are in each of the convolutional filters, $W_{k}$?  What if the kernel size is 4? 5? And the output layer, $W_{out}$?
<p>
<p>
4. Historically NLP models made heavy use of manual feature engineering. In relation to systems with manually engineered features, describe what type of operation is being performed by the convolutional filters.
<p>
5. Suppose that we have two examples, `[foo bar baz]` and `[baz bar foo]`.  Does this model definitely make the same predictions on these? Why or why not?

1. $c^{(i)}_{k}$, 1

  $c_{k}$, 10-（3-1）=8
  
  $\hat{c}_{k}$, 1
  
  and $\hat{C}$. 128
  
2. 10，  1

3. 3x10, 4x10, 5x10; 7x128=896


## (b) Implementing the CNN Model

We'll implement our CNN model below. Our implementation will differ from [Kim (2014)](https://www.aclweb.org/anthology/D14-1181) in that we will support using multiple dense hidden layers after the convolutional layers.

**Before you start**, be sure to answer the short-answer questions above!

In [2]:
import sst

# Load SST dataset
ds = sst.SSTDataset(V=20000).process(label_scheme="binary")
max_len = 40
train_x, train_ns, train_y = ds.as_padded_array('train', max_len=max_len, root_only=True)
dev_x,   dev_ns,   dev_y   = ds.as_padded_array('dev',   max_len=max_len, root_only=True)
test_x,  test_ns,  test_y  = ds.as_padded_array('test',  max_len=max_len, root_only=True)


Loading SST from data/sst/trainDevTestTrees_PTB.zip
Training set:     8,544 trees
Development set:  1,101 trees
Test set:         2,210 trees
Building vocabulary - 16,474 words
Processing to phrases...  Done!
Splits: train / dev / test : 98,794 / 13,142 / 26,052


In [6]:
# Specify model hyperparameters.
epochs = 10
embed_dim = 5
num_filters = [2, 2, 2]
kernel_sizes = [2, 3, 4]
dense_layer_dims = [10, 4]
dropout_rate = 0.7
num_classes = len(ds.target_names)

# Construct the convolutional neural network.
# The form of each keras layer function is as follows:
#    result = keras.layers.LayerType(arguments for the layer)(layer(s) it should use as input)
# concretely,
#    this_layer_output = keras.layers.Dense(100, activation='relu')(prev_layer_vector)
# performs this_layer_output = relu(prev_layer_vector x W + b) where W has 100 columns.

# Input is a special "layer".  It defines a placeholder that will be overwritten by the training data.
# In our case, we are accepting a list of wordids (padded out to max_len).
wordids = keras.layers.Input(shape=(max_len,))

# Embed the wordids.
# Recall, this is just a mathematically equivalent operation to a linear layer and a one-hot
h = keras.layers.Embedding(ds.vocab.size, embed_dim, input_length=max_len)(wordids)

# Construct "filters" randomly initialized filters with dimension "kernel_size" for each size of filter we want.
# With the default hyperparameters, we construct 2 filters each of size 2, 3, 4.  As in the image above, each filter
# is wide enough to span the whole word embedding (this is why the convolution is "1d" as seen in the
# function name below).
conv_layers_for_all_kernel_sizes = []
for kernel_size, filters in zip(kernel_sizes, num_filters):
    conv_layer = keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, activation='relu')(h)
    conv_layer = keras.layers.GlobalMaxPooling1D()(conv_layer)
    conv_layers_for_all_kernel_sizes.append(conv_layer)

# Concat the feature maps from each different size.
h = keras.layers.concatenate(conv_layers_for_all_kernel_sizes, axis=1)

# Dropout can help with overfitting (improve generalization) by randomly 0-ing different subsets of values
# in the vector.
# See https://www.cs.toronto.edu/~hinton/absps/JMLRdropout.pdf for details.
h = keras.layers.Dropout(rate=dropout_rate)(h)

### YOUR CODE HERE
# Add a fully connected layer for each dense layer dimension in dense_layer_dims.
for dim in dense_layer_dims:
    h = keras.layers.Dense(dim, activation='relu')(h)
       
### END YOUR CODE

prediction = keras.layers.Dense(num_classes, activation='softmax')(h)

model = keras.Model(inputs=wordids, outputs=prediction)
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  # From information theory notebooks.
              metrics=['accuracy'])        # What metric to output as we train.

In [7]:
model.reset_states()
model.fit(train_x, train_y, epochs=epochs)

Epoch 1/10
217/217 [==============================] - 3s 3ms/step - loss: 0.6931 - accuracy: 0.5121
Epoch 2/10
217/217 [==============================] - 1s 3ms/step - loss: 0.6870 - accuracy: 0.5228
Epoch 3/10
217/217 [==============================] - 1s 3ms/step - loss: 0.6646 - accuracy: 0.5885
Epoch 4/10
217/217 [==============================] - 1s 3ms/step - loss: 0.6219 - accuracy: 0.6655
Epoch 5/10
217/217 [==============================] - 1s 3ms/step - loss: 0.5768 - accuracy: 0.7094
Epoch 6/10
217/217 [==============================] - 1s 3ms/step - loss: 0.5520 - accuracy: 0.7263
Epoch 7/10
217/217 [==============================] - 1s 3ms/step - loss: 0.5043 - accuracy: 0.7636
Epoch 8/10
217/217 [==============================] - 1s 3ms/step - loss: 0.4832 - accuracy: 0.7725
Epoch 9/10
217/217 [==============================] - 1s 3ms/step - loss: 0.4554 - accuracy: 0.7867
Epoch 10/10
217/217 [==============================] - 1s 3ms/step - loss: 0.4515 - accuracy: 0.7899

In [8]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 40)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 40, 5)        82370       input_1[0][0]                    
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 39, 2)        22          embedding[0][0]                  
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 38, 2)        32          embedding[0][0]                  
______________________________________________________________________________________________

## Evaluation

Call [evaluate](https://keras.io/models/model/#evaluate) on your model.

In [9]:
#### YOUR CODE HERE ####
print("Evaluate on test data")
results = model.evaluate(test_x, test_y)
print("test loss, test acc:", results)
#### END(YOUR CODE) ####

Evaluate on test data
57/57 [==============================] - 0s 1ms/step - loss: 0.5979 - accuracy: 0.7051
test loss, test acc: [0.5978780388832092, 0.7051070928573608]


# Part (c): Tuning Your Model

We'll once again want to optimize hyperparameters for our model to see if we can improve performance. The CNN model includes a number of new parameters that can significantly influence model performance.

In this section, you will be asked to describe the new parameters as well as use them to attempt to improve the performance of your model.

## Part (c) Short Answer Questions

  1. Choose two parameters unique the CNN model, perform at least 10 runs with different combinations of values for these parameters, and then report the dev set results below. ***Hint: Consider wrapping the training code above in a for loop the examines the different values.***  To do this efficiently, you should consider [this paper](http://www.jmlr.org/papers/volume13/bergstra12a/bergstra12a.pdf) from Bergstra and Bengio.  [This blog post](https://blog.floydhub.com/guide-to-hyperparameters-search-for-deep-learning-models/) also has a less formal treatment of the same topic.
  2. Describe any trends you see in experiments above (e.g., can you identify good ranges for the individual parameters; are there any interesting interactions?)
  3. Pick the three best configurations according to the dev set and evaluate them on the test data. Is the ranking of the three best models the same on the dev and test sets?
  4. What was the best accuracy you achieved on the test set?

In [12]:
def tune_cnn(num_filters, kernel_sizes):
    '''
    train model with designed filters nubmer, kernel size, return the accuracy on dev set and test set
    '''
    epochs = 10
    embed_dim = 5
    #num_filters = [2, 2, 2]
    #kernel_sizes = [2, 3, 4]
    dense_layer_dims = [10, 4]
    dropout_rate = 0.7
    num_classes = len(ds.target_names)

    wordids = keras.layers.Input(shape=(max_len,))

    h = keras.layers.Embedding(ds.vocab.size, embed_dim, input_length=max_len)(wordids)

    conv_layers_for_all_kernel_sizes = []
    for kernel_size, filters in zip(kernel_sizes, num_filters):
        conv_layer = keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, activation='relu')(h)
        conv_layer = keras.layers.GlobalMaxPooling1D()(conv_layer)
        conv_layers_for_all_kernel_sizes.append(conv_layer)

    # Concat the feature maps from each different size.
    h = keras.layers.concatenate(conv_layers_for_all_kernel_sizes, axis=1)

    h = keras.layers.Dropout(rate=dropout_rate)(h)

    for dim in dense_layer_dims:
        h = keras.layers.Dense(dim, activation='relu')(h)
       

    prediction = keras.layers.Dense(num_classes, activation='softmax')(h)

    model = keras.Model(inputs=wordids, outputs=prediction)
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',  # From information theory notebooks.
                  metrics=['accuracy'])        # What metric to output as we train.

    model.reset_states()
    model.fit(train_x, train_y, epochs=epochs, validation_data=(dev_x, dev_y))


    print("Evaluate on dev data")
    dev_score = model.evaluate(dev_x, dev_y)
    print("Dev loss, test acc:", dev_score)
    dev_acc.append(dev_score[1])  # dev accuracy 


    print("Evaluate on test data")
    test_score = model.evaluate(test_x, test_y)
    print("test loss, test acc:", test_score)
    test_acc.append(test_score[1])  # test accuracy 
    
    return dev_acc, test_acc

In [57]:
num_filters_list = [[2, 2, 2],[4, 4, 4], [8, 8, 8], [16, 16, 16]]  # filter number range

kernel_sizes_list = [[2, 2, 2], [2, 3, 4], [3, 4, 5], [4, 5, 6]]  # kernel size range

dev_acc = []
test_acc = []
num_paras = []
kernel_paras = []

for num_filters in num_filters_list:
    for kernel_sizes in kernel_sizes_list:
        
        num_paras.append(num_filters)
        kernel_paras.append(kernel_sizes)
        
        print('Number of filters: ', num_filters)
        print('Kernel sizes: ', kernel_sizes)
        a, b = tune_cnn(num_filters, kernel_sizes)
        dev_acc.extend(a)
        test_acc.extend(b)

Number of filters:  [2, 2, 2]
Kernel sizes:  [2, 2, 2]
Epoch 1/10
217/217 [==============================] - 1s 4ms/step - loss: 0.6928 - accuracy: 0.5186 - val_loss: 0.6924 - val_accuracy: 0.5092
Epoch 2/10
217/217 [==============================] - 1s 3ms/step - loss: 0.6883 - accuracy: 0.5258 - val_loss: 0.6852 - val_accuracy: 0.5975
Epoch 3/10
217/217 [==============================] - 1s 3ms/step - loss: 0.6705 - accuracy: 0.5762 - val_loss: 0.6563 - val_accuracy: 0.6823
Epoch 4/10
217/217 [==============================] - 1s 3ms/step - loss: 0.6361 - accuracy: 0.6430 - val_loss: 0.6246 - val_accuracy: 0.7007
Epoch 5/10
217/217 [==============================] - 1s 3ms/step - loss: 0.5968 - accuracy: 0.7104 - val_loss: 0.6046 - val_accuracy: 0.7076
Epoch 6/10
217/217 [==============================] - 1s 3ms/step - loss: 0.5791 - accuracy: 0.7117 - val_loss: 0.5808 - val_accuracy: 0.7282
Epoch 7/10
217/217 [==============================] - 1s 3ms/step - loss: 0.5525 - accuracy: 

217/217 [==============================] - 1s 3ms/step - loss: 0.3947 - accuracy: 0.8329 - val_loss: 0.5635 - val_accuracy: 0.7167
Epoch 8/10
217/217 [==============================] - 1s 3ms/step - loss: 0.3804 - accuracy: 0.8436 - val_loss: 0.5822 - val_accuracy: 0.7190
Epoch 9/10
217/217 [==============================] - 1s 6ms/step - loss: 0.3278 - accuracy: 0.8669 - val_loss: 0.6112 - val_accuracy: 0.7156
Epoch 10/10
217/217 [==============================] - 1s 3ms/step - loss: 0.3110 - accuracy: 0.8780 - val_loss: 0.6343 - val_accuracy: 0.7144
Evaluate on dev data
28/28 [==============================] - 0s 1ms/step - loss: 0.6343 - accuracy: 0.7144
Dev loss, test acc: [0.6342650055885315, 0.7144495248794556]
Evaluate on test data
57/57 [==============================] - 0s 1ms/step - loss: 0.5908 - accuracy: 0.7298
test loss, test acc: [0.5908088684082031, 0.7298187613487244]
Number of filters:  [4, 4, 4]
Kernel sizes:  [2, 3, 4]
Epoch 1/10
217/217 [===========================

Epoch 1/10
217/217 [==============================] - 2s 4ms/step - loss: 0.6947 - accuracy: 0.4947 - val_loss: 0.6916 - val_accuracy: 0.5092
Epoch 2/10
217/217 [==============================] - 1s 4ms/step - loss: 0.6890 - accuracy: 0.5269 - val_loss: 0.6808 - val_accuracy: 0.5367
Epoch 3/10
217/217 [==============================] - 1s 3ms/step - loss: 0.6574 - accuracy: 0.6180 - val_loss: 0.5660 - val_accuracy: 0.7362
Epoch 4/10
217/217 [==============================] - 1s 4ms/step - loss: 0.5239 - accuracy: 0.7630 - val_loss: 0.5158 - val_accuracy: 0.7729
Epoch 5/10
217/217 [==============================] - 1s 4ms/step - loss: 0.4049 - accuracy: 0.8393 - val_loss: 0.5371 - val_accuracy: 0.7569
Epoch 6/10
217/217 [==============================] - 1s 4ms/step - loss: 0.3210 - accuracy: 0.8725 - val_loss: 0.5805 - val_accuracy: 0.7615
Epoch 7/10
217/217 [==============================] - 1s 4ms/step - loss: 0.2639 - accuracy: 0.9033 - val_loss: 0.6340 - val_accuracy: 0.7569
Epoch 

217/217 [==============================] - 1s 4ms/step - loss: 0.1259 - accuracy: 0.9598 - val_loss: 0.7086 - val_accuracy: 0.7649
Epoch 8/10
217/217 [==============================] - 1s 4ms/step - loss: 0.0933 - accuracy: 0.9688 - val_loss: 0.8626 - val_accuracy: 0.7569
Epoch 9/10
217/217 [==============================] - 1s 4ms/step - loss: 0.0805 - accuracy: 0.9763 - val_loss: 0.9207 - val_accuracy: 0.7580
Epoch 10/10
217/217 [==============================] - 1s 4ms/step - loss: 0.0742 - accuracy: 0.9795 - val_loss: 1.0143 - val_accuracy: 0.7580
Evaluate on dev data
28/28 [==============================] - 0s 1ms/step - loss: 1.0143 - accuracy: 0.7580
Dev loss, test acc: [1.0142877101898193, 0.7580274939537048]
Evaluate on test data
57/57 [==============================] - 0s 1ms/step - loss: 0.8972 - accuracy: 0.7776
test loss, test acc: [0.8971751928329468, 0.7775947451591492]
Number of filters:  [16, 16, 16]
Kernel sizes:  [3, 4, 5]
Epoch 1/10
217/217 [========================

In [60]:
print('list of models performance')

for i in range(len(dev_acc)):
    print('Filters num {0}, Kernel size {1} -- Dev Accuracy: {2:.2%}, Test Accuracy: {3:.2%}'.format(
        num_paras[i], kernel_paras[i], dev_acc[i], test_acc[i]))


list of models performance
Filters num [2, 2, 2], Kernel size [2, 2, 2] -- Dev Accuracy: 72.25%, Test Accuracy: 71.77%
Filters num [2, 2, 2], Kernel size [2, 3, 4] -- Dev Accuracy: 69.84%, Test Accuracy: 68.64%
Filters num [2, 2, 2], Kernel size [3, 4, 5] -- Dev Accuracy: 73.85%, Test Accuracy: 75.34%
Filters num [2, 2, 2], Kernel size [4, 5, 6] -- Dev Accuracy: 73.05%, Test Accuracy: 71.99%
Filters num [4, 4, 4], Kernel size [2, 2, 2] -- Dev Accuracy: 71.44%, Test Accuracy: 72.98%
Filters num [4, 4, 4], Kernel size [2, 3, 4] -- Dev Accuracy: 72.94%, Test Accuracy: 71.72%
Filters num [4, 4, 4], Kernel size [3, 4, 5] -- Dev Accuracy: 74.54%, Test Accuracy: 75.67%
Filters num [4, 4, 4], Kernel size [4, 5, 6] -- Dev Accuracy: 73.39%, Test Accuracy: 72.87%
Filters num [8, 8, 8], Kernel size [2, 2, 2] -- Dev Accuracy: 76.26%, Test Accuracy: 76.06%
Filters num [8, 8, 8], Kernel size [2, 3, 4] -- Dev Accuracy: 75.11%, Test Accuracy: 76.77%
Filters num [8, 8, 8], Kernel size [3, 4, 5] -- Dev A

In [62]:
top_dev_config = np.argsort(dev_acc)[:-4:-1]
top_test_config = np.argsort(test_acc)[:-4:-1]

print("According to the dev set")
for i in top_dev_config:
    print('Filters num {0}, Kernel size {1} -- Dev Accuracy: {2:.3%}, Test Accuracy: {3:.3%}'.format(
        num_paras[i], kernel_paras[i], dev_acc[i], test_acc[i]))
    
print()

print("According to the test set")
for i in top_test_config:
    print('Filters num {0}, Kernel size {1} -- Dev Accuracy: {2:.3%}, Test Accuracy: {3:.3%}'.format(
        num_paras[i], kernel_paras[i], dev_acc[i], test_acc[i]))

According to the dev set
Filters num [16, 16, 16], Kernel size [2, 2, 2] -- Dev Accuracy: 77.408%, Test Accuracy: 76.606%
Filters num [8, 8, 8], Kernel size [4, 5, 6] -- Dev Accuracy: 77.179%, Test Accuracy: 76.332%
Filters num [16, 16, 16], Kernel size [4, 5, 6] -- Dev Accuracy: 76.720%, Test Accuracy: 75.453%

According to the test set
Filters num [16, 16, 16], Kernel size [2, 3, 4] -- Dev Accuracy: 75.803%, Test Accuracy: 77.759%
Filters num [8, 8, 8], Kernel size [2, 3, 4] -- Dev Accuracy: 75.115%, Test Accuracy: 76.771%
Filters num [16, 16, 16], Kernel size [2, 2, 2] -- Dev Accuracy: 77.408%, Test Accuracy: 76.606%
